# Feature Engineering
The main problem in this data set is that there are only 30 observations, and as of this far, 20 agro-climatic indicators likely with high collinearity. To deal with this problem the following steps will be applied in this part:
* Using simple techniques and domain knowledge only some of the most important columns will be selected as predictor variables. 
* Each of the four states' data will be stacked on top of each other. This data structure is called **panel data**<br>
    **The advantages of this method:**
    1. Sample size will increase to 120. <br>
    2. We can split the data to train and test and still do cross validation with the train data
    **The disadvantages of this method:**
     1. Since the each states' climate data is annually sampled, spatially averaged and detrended they are similar to each other ( see the plot below). Thus stacking them on top of each other is similar, to an extend, to repeating the 30 year-observation 4 times in the same data. The difference between the states could be from noise (impossible to model per se), difference in the affect of the climate variables on soybeans yield in each state. The former can be implemented in a model by one-hot encoding the states. The similarities between the observations at each states can be implented as autocorrelations with lag=30yr. This requires special regression methods such as mixed effect model. However, the lenght of the data we have is too short for such a complex model. 
     2. One-hot encoding will create 3 more features. Since the climate variables are similar in each state as explained above modeling the difference is difficult and normally requires large sample size. 

     Despite these drawbacks, the benefits of creating panel data outwiegh the downsides.
  

In [25]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from scipy.stats import norm, skew #for some statistics
from scipy import stats

from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from statsmodels.tsa.tsatools import detrend

In [22]:
# Read the all variables data from the data wrangling steps
df = pd.read_pickle('Data/all_variables.pkl')

#### Splitting data to train and test
The rest of the parts require carefully separating train and validation data. Otherwise significant data leakage could occur. Train and test splitting will be done by manually without using builtin sklearn or other packages because of the panel structure of the data.

In [23]:
# Creat a function to split data for train and test and get dummies for states
def panel_data_split(df,y_column, test_size, one_hot_encoding=True):
    # Collect features and response columns from each state group in a list
    train_collect = []
    test_collect = []
    # loop over the soy_states and split train and test for each state
    for state in soy_states:
        # Select the state rows
        df_state = df.query('State == @state').copy()
        # split train and test for the state
        df_state_train = df_state[:-test_size]
        df_state_test = df_state[-test_size:]
        # append the train, test dataframe for the state in a list
        train_collect.append(df_state_train)
        test_collect.append(df_state_test)
    # concat the state train, test splits on axis=0
    df_train = pd.concat(train_collect, axis=0)
    df_test = pd.concat(test_collect, axis=0)
    # get dummies for states
    if one_hot_encoding == True:
        df_train = pd.get_dummies(df_train, prefix="", 
                                  prefix_sep="", drop_first=True)
        df_test = pd.get_dummies(df_test, prefix="", 
                                  prefix_sep="", drop_first=True)
    # Split the features and response columns
    X_train = df_train.drop(y_column, axis=1).reset_index(drop=True)
    y_train = df_train[y_column].reset_index(drop=True)
    X_test = df_test.drop(y_column, axis=1).reset_index(drop=True)
    y_test = df_test[y_column].reset_index(drop=True)
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = panel_data_split(df, y_column='Yield',test_size=5, one_hot_encoding=False)
df_train_all = pd.concat([X_train, y_train], axis=1)
df_test_all = pd.concat([X_test, y_test], axis=1)

Let's check the correlation matrix to decide which variables to conider for modelling. Note that only the train data is analysed in order not to cause data leakage for reliable validation.

In [26]:
# let's plot pairwise correlation heatmap
f, ax = plt.subplots(figsize=(14, 10))
ax = sns.heatmap(df_train_all.corr(), annot=True, cmap='PuOr')
plt.show()

Let's recall what these variables
<a id='clim_ind_names'></a>

Let's drop some highly collinear variables, and some based on domain knowledge. For instance heavy and very heavy precipitation (R10mm & R20mm) could provide information on crop damage and run off loss. However, in this data it seems like both have a positive effect on crop yield. This is probably because of the annual aggregation ( actually growing season aggregation at each year) and spatial averaging over a large area which tends to mask the actual effect of the local events such as this. Furthermore, all the rain variables are strongly correlated to each other, this means when number of R20mm events is high at a certain year, RR1mm is very likely high too which is one of the key factor for higher yield. The other variables are dropped based on similar fashion.

In [38]:
vars_to_drop = ["BEDD", "DTR", "ID", "R10mm", "R20mm", "RR", "SU", "TR", "TX", "WW", "CSDI", "CFD"]
df_train = df_train_all.copy()
df_test = df_test_all.copy()
for var in df_train_all:
    if var in vars_to_drop:
        df_train = df_train.drop(var, axis=1)
        df_test = df_test.drop(var, axis=1)
df_train.head()

time        FD        RR1        TG       CDD        CSU       CWD  \
0 1981-01-01 -2.397412   9.262426 -0.322938 -1.754717  -2.925107  0.217959   
1 1982-01-01  1.847563   1.751770 -0.026335 -1.452061 -13.601458 -1.749869   
2 1983-01-01 -5.418393  -9.351781  0.928018  0.301343  18.464665 -2.145723   
3 1984-01-01  1.353240   0.678740  0.089718 -1.139041   5.500659  0.849960   
4 1985-01-01 -5.690858  12.226350 -0.119721 -1.225932  -7.104322  0.410591   

       WSDI        GSL     State     Yield  
0 -2.088179  20.885830  ILLINOIS  3.198925  
1 -2.047021  -6.635000  ILLINOIS  3.241528  
2  3.359101  16.380019  ILLINOIS -6.215869  
3 -1.964704   4.426492  ILLINOIS -4.673267  
4 -1.923546  -9.385414  ILLINOIS  5.869336

In [39]:
# Let's see how strong the colinearity in overall data
np.linalg.det(df_train.corr())

0.06929323307659126

The determinant of entire correlation matrix can be used as a proxy for how strong colinearity in the data. The closer the determinant to 1 is better (less colinearity). We have got a result indicating there is still some colinearity in the data. 

#### Checking Normality of the Variables.
Regression models assume that the variables are normally distributed. Therefore the following steps will be applied to check for distribution of the variables and transform those with high skeweness and kurtosis distributions:
* Do skewness test for each columns.
* Consider those with absolute value of skewness > 0.5 as skewed the rest as acceptable
* Apply box-cox transformation 

In [40]:
# Skewness test
skewed = df_train.drop('State', axis=1).skew().sort_values(ascending=False)
pd.DataFrame({'skewness': skewed})

skewness
WSDI   2.363110
CDD    1.575828
CWD    0.603883
GSL    0.340316
FD     0.332010
CSU    0.293348
TG    -0.321851
RR1   -0.520367
Yield -0.905147

Several variables are skewed. We can fix the skewness by applying power transformation to variables including the response variable. 

In [41]:
# Apply yeo-johnson power transformation whcih is unlike box-cox method allows negative values
pt = PowerTransformer(method='yeo-johnson', standardize=True)
df_pt_train = pt.fit_transform(df_train.drop(['State', 'time'], axis=1))
df_pt_train = pd.DataFrame(df_pt_train, columns=
                         df_train.drop(['State', 'time'], axis=1).columns)
df_pt_train.index = df_train.index
df_pt_train['time'] = df_train['time']
df_pt_train['State'] = df_train['State']
pd.DataFrame(df_pt_train.skew().sort_values(ascending=False), columns=['skewness'])

skewness
FD     0.143554
GSL    0.142473
RR1    0.130090
Yield  0.036550
CWD    0.007545
CSU   -0.002498
WSDI  -0.010414
TG    -0.027553
CDD   -0.105953

In [42]:
# Apply yeo-johnson power transformation whcih is unlike box-cox method allows negative values
df_pt_test = pt.transform(df_test.drop(['State', 'time'], axis=1))
df_pt_test = pd.DataFrame(df_pt_test, columns=
                         df_test.drop(['State', 'time'], axis=1).columns)
df_pt_test.index = df_test.index
df_pt_test['time'] = df_test['time']
df_pt_test['State'] = df_test['State']
pd.DataFrame(df_pt_test.skew().sort_values(ascending=False), columns=['skewness'])

skewness
TG     0.614859
Yield  0.222808
WSDI   0.160802
RR1    0.108816
GSL   -0.505981
CDD   -0.528688
CWD   -0.530414
CSU   -0.666650
FD    -0.667629

Some variables are a bit skewed in the test data, but notice that the size of the test data is rather small, only 20 sample. Even if the population was exactly normal, drawing 20 sample from this population would be likely to result slighlty skewed sample distribution. The same is true for the train data which has 100 samples, but this study about the methodology. The sample data can be extended in the future by calculating the agro-climatic indicators manually from fundemental climate variables which usually have larger sample size. Once a larger sample is obtained this notebook can be used as it is to built more reliable models.

In [43]:
X_train = df_pt_train.drop(['Yield','time'], axis=1)
X_train = pd.get_dummies(X_train, prefix="", 
                                  prefix_sep="", drop_first=True)
y_train = df_pt_train['Yield'] # keeping time for easy cross validation

X_test = df_pt_test.drop(['Yield','time'], axis=1)
X_test = pd.get_dummies(X_test, prefix="", 
                                  prefix_sep="", drop_first=True)
y_test = df_pt_test['Yield']

Now that all the variables are standardized and tranformed to normal distribution it is a good idea to revisit the colinearity check since pearson correlations are more reliable when the variables are normally distributed.

In [44]:
# Let's see how strong the colinearity in overall data
np.linalg.det(df_pt_train.drop('State', axis=1).corr())

0.07906721661567707

The colinearity is only slightly reduced. Some more variables could be droped, but I will leave that to Lasso regression. And PCA can handle colinearity. 